In [3]:
import os
import sys

PROJECT_ROOT = os.path.abspath("..") # 상위 폴더의 절대 경로 얻음 
sys.path.insert(0, PROJECT_ROOT) #모듈 경로를 강제로 추가해서 import 에러 방지

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
from src.config import MART_COLUMNS_PATH, TPS_YN_PATH,VOD_LOG_SAMPLE_PATH ,IS_CANCELED, TV_SUB, INTERNET_SUB,AGE_DECADE,IS_AGE_DECADE
from src.stats_test import chisquare_test, compare_two_groups
from scipy.stats import chi2_contingency
from src.eda_utils import plot_crosstab
rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

In [6]:
# 데이터 로드
user = pd.read_pickle(MART_COLUMNS_PATH)
vod  = pd.read_pickle(TPS_YN_PATH)
log  = pd.read_pickle(VOD_LOG_SAMPLE_PATH)

In [4]:
# 1️⃣ 층화 샘플링
analysis_df = (
    user
    .groupby(IS_CANCELED, group_keys=False)
    .apply(lambda x: x.sample(n=100_000, random_state=42))
)
def get_sub_type(tv, internet):
    if tv == 1 and internet == 0:
        return 'TV_ONLY'
    elif tv == 1 and internet == 1:
        return 'TV+INTERNET'
    else:
        return None

analysis_df['sub_type'] = analysis_df.apply(
    lambda x: get_sub_type(x[TV_SUB], x[INTERNET_SUB]), axis=1
)



C:\Users\dkswn\AppData\Local\Temp\ipykernel_15480\1300794528.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(IS_CANCELED, group_keys=False)
C:\Users\dkswn\AppData\Local\Temp\ipykernel_15480\1300794528.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=100_000, random_state=42))


귀무가설(HO)

대립가설(H1)


In [44]:
result = chisquare_test(
    analysis_df,
    col1='sub_type',
    col2=IS_CANCELED
)

print(f"Chi-square statistic: {result['statistic']:.2f}")
print(f"p-value: {result['p_value']:.4f}")
print(f"자유도(dof): {result['dof']}")


plot_crosstab(
    analysis_df,
    'sub_type',
    IS_CANCELED,
    '../outputs/figures/sub_type_cancel_ratio.png',
    '../outputs/tables/sub_type_cancel_ratio.csv'
)

chi_result_df = pd.DataFrame([{
    'chi2_statistic': result['statistic'],
    'p_value': result['p_value'],
    'dof': result['dof']
}])
chi_result_df.to_csv('../outputs/tables/chi2_result.csv', index=False)


귀무가설 (H₀)
- 연령대에 따라 해지율에 차이가 없다.

대립가설 (H₁)
- 연령대에 따라 해지율에 차이가 있다.

	​


In [10]:
user[AGE_DECADE] = user[AGE_DECADE].astype(str)
age_cancel_sample = user[user[AGE_DECADE] != '연령없음'].copy()

# 해지 여부 숫자 변환
age_cancel_sample[IS_CANCELED] = age_cancel_sample[IS_CANCELED].map({'유지':0, '해지':1})

In [11]:
# 카이제곱 검정
result = chisquare_test(
    age_cancel_sample,
    col1=AGE_DECADE,
    col2=IS_CANCELED
)

print(f"Chi-square statistic: {result['statistic']:.2f}")
print(f"p-value: {result['p_value']:.6f}")
print(f"Degrees of freedom: {result['dof']}")

Chi-square statistic: 161868.63
p-value: 0.000000
Degrees of freedom: 8


귀무가설 (H₀)
- 마케팅 수신 동의 여부가 해지율에 차이가 없다.

대립가설 (H₁)
- 마케팅 수신 동의 여부가 해지율에 차이가 있다.

In [7]:
# H2: 마케팅 수신 동의 여부가 해지율과 관련 있다
marketing_consent_list = [
    "EMAIL_RECV_CLS_NM",
    "SMS_SEND_CLS_NM"
]


In [9]:
h2_results = []

for col in marketing_consent_list:
    result = chisquare_test(
        df=vod,
        col1=col,
        col2=IS_CANCELED
    )

    h2_results.append({
        "variable": col,
        "chi2": result["statistic"],
        "p_value": result["p_value"],
        "dof": result["dof"]
    })

h2_result_df = pd.DataFrame(h2_results)
h2_result_df


,variable,chi2,p_value,dof
0,EMAIL_RECV_CLS_NM,2621.557135,0.0,3
1,SMS_SEND_CLS_NM,4167.035071,0.0,3
